# Use examples of [edges](https://github.com/romainsacchi/edges)

Author: [romainsacchi](https://github.com/romainsacchi)

This notebook shows examples on how to use `edge` to use exchange-specific
characterization factors in the characterization matrix of `bw2calc`.

## Requirements

* **Pyhton 3.10 or higher (up to 3.11) is highly recommended**

# Use case with [brightway2](https://brightway.dev/)

`brightway2` is an open source LCA framework for Python.
To use `premise` from `brightway2`, it requires that you have an activated `brightway2` project with a `biosphere3` database as well as an [ecoinvent](https://ecoinvent.prg) v.3 cut-off or consequential database registered in that project. Please refer to the brightway [documentation](https://brightway.dev) if you do not know how to create a project and install ecoinvent.

In [1]:
from edges import EdgeLCIA, get_available_methods
import bw2data

One can simply build its own LCIA file.
Let's consider the following LCIA file (saved under `lcia_example_1.json`):

In [2]:
[
  {
    "supplier": {
      "operator": "startswith", # can be "equals", "contains", "startswith"
      "matrix": "biosphere", # can be "biosphere" or "technosphere"
      "name": "Carbon dioxide", # edges will search for any supplier in the biosphere matrix that has a name starting with "Carbon dioxide"
    },
    "consumer": {
      "matrix": "technosphere", # the consumer is in the technosphere matrix
      "type": "process" # the consumer is a process
    },
    "value": 1.0 # the characterization factor
  },
  {
    "supplier": {
      "name": "Methane, fossil",
      "operator": "contains", # different operator
      "matrix": "biosphere"
    },
    "consumer": {
      "matrix": "technosphere",
      "type": "process"
    },
    "value": 28.0
  },
  {
    "supplier": {
      "name": "Dinitrogen monoxide",
      "operator": "equals", # different operator
      "matrix": "biosphere"
    },
    "consumer": {
      "matrix": "technosphere",
      "type": "process"
    },
    "value": 265.0
  }
]

[{'supplier': {'operator': 'startswith',
   'matrix': 'biosphere',
   'name': 'Carbon dioxide'},
  'consumer': {'matrix': 'technosphere', 'type': 'process'},
  'value': 1.0},
 {'supplier': {'name': 'Methane, fossil',
   'operator': 'contains',
   'matrix': 'biosphere'},
  'consumer': {'matrix': 'technosphere', 'type': 'process'},
  'value': 28.0},
 {'supplier': {'name': 'Dinitrogen monoxide',
   'operator': 'equals',
   'matrix': 'biosphere'},
  'consumer': {'matrix': 'technosphere', 'type': 'process'},
  'value': 265.0}]

In [2]:
# activate the bw project
bw2data.projects.set_current("ecoinvent-3.10-cutoff")

In [3]:
act = bw2data.Database("ecoinvent-3.10.1-cutoff").random()
act

'maize grain production' (kilogram, US-WI, None)

In [4]:
LCA = EdgeLCIA(
    demand={act: 1},
    method=("some", "method"),
    filepath="lcia_example_1.json"
)
LCA.lci()

LCA.map_exchanges()

LCA.evaluate_cfs()
LCA.lcia()
LCA.score

/opt/homebrew/Caskroom/miniforge/base/envs/bw2/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 4.83e+13)
  warnings.warn(msg, UmfpackWarning)


Identifying eligible exchanges...


100%|██████████| 3/3 [00:00<00:00, 35.46it/s]


1.6424403416511533

We can print some statistics:

In [5]:
LCA.statistics()

+----------------------+------------------------+
|       Activity       | maize grain production |
|     Method name      |   ('some', 'method')   |
|      Data file       |     lcia_example_1     |
| Unique CFs in method |           3            |
|   Unique CFs used    |           3            |
|  Exc. characterized  |         12788          |
| Exc. uncharacterized |         323121         |
+----------------------+------------------------+


### Generate dataframe of characterization factors used

The `generate_cf_table` method generates a dataframe of the characterization factors used in the calculation. One can see the characterization factors used for each exchange in the system.

In [6]:
df = LCA.generate_cf_table()

In [7]:
# we can see under the "CF" column
# the characterization factors used for each exchange in the system
df

,supplier name,supplier categories,consumer name,consumer reference product,consumer location,amount,CF,impact
0,"Carbon dioxide, non-fossil","(air, urban air close to ground)","heat and power co-generation, wood chips, 6667 kW","heat, district or industrial, other than natur...",IN-OR,1.419378e-08,1.0,1.419378e-08
1,"Carbon dioxide, non-fossil","(air, urban air close to ground)","lath, softwood, raw, kiln drying to u=10%","sawnwood, lath, softwood, raw, dried (u=10%)",CA-QC,8.625774e-09,1.0,8.625774e-09
2,"Carbon dioxide, non-fossil","(air, urban air close to ground)","sulfate pulp production, from softwood, bleached","tall oil, crude",RER,7.287865e-10,1.0,7.287865e-10
3,"Carbon dioxide, non-fossil","(air, urban air close to ground)","treatment of waste wood, untreated, municipal ...","waste wood, untreated",CH,5.306133e-05,1.0,5.306133e-05
4,"Carbon dioxide, non-fossil","(air, urban air close to ground)","biomethane, low pressure burned in micro gas t...","heat, central or small-scale, other than natur...",CH,2.314395e-18,1.0,2.314395e-18
...,...,...,...,...,...,...,...,...
12783,Dinitrogen monoxide,"(air,)","kerosene production, petroleum refinery operation",kerosene,BR,8.170086e-14,265.0,2.165073e-11
12784,Dinitrogen monoxide,"(air,)","transport, freight, lorry 7.5-16 metric ton, E...","transport, freight, lorry 7.5-16 metric ton, E...",RER,4.625460e-11,265.0,1.225747e-08
12785,Dinitrogen monoxide,"(air,)","market for electricity, high voltage","electricity, high voltage",NE,7.565686e-12,265.0,2.004907e-09
12786,Dinitrogen monoxide,"(air,)","fibreboard production, hard","residual wood, dry",RoW,1.651729e-15,265.0,4.377081e-13


In [8]:
df.groupby("supplier name")["CF"].mean()

supplier name
Carbon dioxide, fossil                               1.0
Carbon dioxide, from soil or biomass stock           1.0
Carbon dioxide, in air                               1.0
Carbon dioxide, non-fossil                           1.0
Carbon dioxide, non-fossil, resource correction      1.0
Carbon dioxide, to soil or biomass stock             1.0
Dinitrogen monoxide                                265.0
Methane, fossil                                     28.0
Name: CF, dtype: float64